In [1]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

## 构造一个执行两向量加法的张量程序。
- TVMScript 是一种让我们能以 Python 抽象语法树的形式来表示张量程序的方式。
- 这段代码并不实际对应一个 Python 程序，而是对应一个机器学习编译过程中的张量程序。
- TVMScript 的语言设计是为了与 Python 语法所对应，并在 Python 语法的基础上增加了能够帮助程序分析与变换的额外结构。

In [2]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    # 定义多维数组用于存储变量
    def main(A: T.Buffer[128, "float32"],
             B: T.Buffer[128, "float32"],
             C: T.Buffer[128, "float32"]):
        # extra annotations for the function
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i in range(128):
            with T.block("C"):
                # declare a data parallel iterator on spatial domain
                vi = T.axis.spatial(128, i)
                C[vi] = A[vi] + B[vi]

- MyModule 是 IRModule 数据结构的一个实例，是一组张量函数的集合。
- 我们可以通过 script 函数得到这个 IRModule 的 TVMScript 表示。这个函数对于在一步步程序变换间检查 IRModule 而言非常有帮助。

In [10]:
print(type(MyModule))
print(MyModule.script())

<class 'tvm.ir.module.IRModule'>
@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[128, "float32"], B: tir.Buffer[128, "float32"], C: tir.Buffer[128, "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i in tir.serial(128):
            with tir.block("C"):
                vi = tir.axis.spatial(128, i)
                tir.reads(A[vi], B[vi])
                tir.writes(C[vi])
                C[vi] = A[vi] + B[vi]
    


在任何时刻，我们都可以通过 build 将一个 IRModule 转化为可以执行的函数。

In [11]:
rt_mod = tvm.build(MyModule, target="llvm") # cpu
type(rt_mod)

tvm.driver.build_module.OperatorModule

在编译后，mod 包含了一组可以执行的函数。我们可以通过这些函数的名字拿到对应的函数。

In [12]:
func = rt_mod["main"]
func

In [13]:
a = tvm.nd.array(np.arange(128, dtype="float32"))
b = tvm.nd.array(np.ones(128, dtype="float32"))
c = tvm.nd.array(np.empty((128,), dtype="float32"))

In [14]:
print(a, b, c)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [15]:
func(a, b, c)

In [16]:
print(c)

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.
  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.
  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.
  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.
  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.
  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112.
 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126.
 127. 128.]


## 张量程序变换
现在我们开始变换张量程序。一个张量程序可以通过一个辅助的名为调度（schedule）的数据结构得到变换。

In [17]:
sch = tvm.tir.Schedule(MyModule)
type(sch)

tvm.tir.schedule.schedule.Schedule

我们首先尝试拆分循环。

In [18]:
block_c = sch.get_block("C")
(i,) = sch.get_loops(block_c)
i_0, i_1, i_2 = sch.split(i, factors=[None, 4, 4])
print(sch.mod.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[128, "float32"], B: tir.Buffer[128, "float32"], C: tir.Buffer[128, "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i_0, i_1, i_2 in tir.grid(8, 4, 4):
            with tir.block("C"):
                vi = tir.axis.spatial(128, i_0 * 16 + i_1 * 4 + i_2)
                tir.reads(A[vi], B[vi])
                tir.writes(C[vi])
                C[vi] = A[vi] + B[vi]
    


我们可以对这些循环重新排序。现在我们将 i_2 移动到 i_1 的外侧。

In [19]:
sch.reorder(i_0, i_2, i_1)
print(sch.mod.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[128, "float32"], B: tir.Buffer[128, "float32"], C: tir.Buffer[128, "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i_0, i_2, i_1 in tir.grid(8, 4, 4):
            with tir.block("C"):
                vi = tir.axis.spatial(128, i_0 * 16 + i_1 * 4 + i_2)
                tir.reads(A[vi], B[vi])
                tir.writes(C[vi])
                C[vi] = A[vi] + B[vi]
    


最后，我们可以标注我们想要并行最外层的循环。

In [20]:
sch.parallel(i_0)
print(sch.mod.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[128, "float32"], B: tir.Buffer[128, "float32"], C: tir.Buffer[128, "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i_0 in tir.parallel(8):
            for i_2, i_1 in tir.grid(4, 4):
                with tir.block("C"):
                    vi = tir.axis.spatial(128, i_0 * 16 + i_1 * 4 + i_2)
                    tir.reads(A[vi], B[vi])
                    tir.writes(C[vi])
                    C[vi] = A[vi] + B[vi]
    


In [21]:
transformed_mod = tvm.build(sch.mod, target="llvm")  # The module for CPU backends.
transformed_mod["main"](a, b, c)
print(c)

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.
  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.
  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.
  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.
  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.
  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112.
 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126.
 127. 128.]


 ## 通过张量表达式（Tensor Expression，TE）构造张量程序
 在之前的例子中，我们直接使用 TVMScript 构造张量程序。在实际中，通过现有的定义方便地构造这些函数是很有帮助的。张量表达式（tensor expression）是一个帮助我们将一些可以通过表达式表示的张量计算转化为张量程序的 API。

In [22]:
from tvm import te

# declare the c omputation using the expression API
A = te.placeholder((128,), name="A")
B = te.placeholder((128,), name="B")
C = te.compute((128,), lambda i: A[i] + B[i], name="C")

# create a function with the specified list of arguments.
func = te.create_prim_func([A, B, C])
func = func.with_attr("global_symbol", "main")
ir_mod_from_te = IRModule({"main": func})
print(ir_mod_from_te.script())

@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[128, "float32"], B: tir.Buffer[128, "float32"], C: tir.Buffer[128, "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i0 in tir.serial(128):
            with tir.block("C"):
                i = tir.axis.spatial(128, i0)
                tir.reads(A[i], B[i])
                tir.writes(C[i])
                C[i] = A[i] + B[i]
    


## 变换一个矩阵乘法程序

在上面的例子中，我们展示了如何变换一个向量加法程序。现在我们尝试应用一些变换到一个稍微更复杂的的程序——矩阵乘法程序。我们首先使用张量表达式 API 构造初始的张量程序，并编译执行它。

In [23]:
from tvm import te

M = 1024
N = 1024
K = 1024

dtype = "float32"
target = "llvm"
dev = tvm.device(target, 0)
number = 1

k = te.reduce_axis((0, K), name="k")
A = te.placeholder((M, K), name="A")
B = te.placeholder((K, N), name="B")
C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")

# Default
func = te.create_prim_func([A, B, C])
func = func.with_attr("global_symbol", "main")
ir_module = tvm.IRModule({"main": func})
print(type(func))
print(func)
print(type(ir_module))
print(ir_module.script())

# ------- 编译 -------

func = tvm.build(ir_module, target=target)

# ------- 执行 -------

a = tvm.nd.array(np.random.randn(M, K).astype(dtype), dev)
b = tvm.nd.array(np.random.randn(K, N).astype(dtype), dev)
c = tvm.nd.array(np.empty((M, K), dtype=dtype), dev)

func(a, b, c)

evaluator = func.time_evaluator(func.entry_name, dev, number=number)
print("Baseline: %f" % evaluator(a, b, c).mean)

<class 'tvm.tir.function.PrimFunc'>
primfn(var_A: handle, var_B: handle, var_C: handle) -> ()
  attr = {"global_symbol": "main", "tir.noalias": True}
  buffers = {A: Buffer(A_1: Pointer(global float32), float32, [1024, 1024], []),
             B: Buffer(B_1: Pointer(global float32), float32, [1024, 1024], []),
             C: Buffer(C_1: Pointer(global float32), float32, [1024, 1024], [])}
  buffer_map = {var_A: A, var_B: B, var_C: C} {
  block([], "root") {
    tir.reads([])
    tir.writes([])
    for (i0: int32, 0, 1024) {
      for (i1: int32, 0, 1024) {
        for (i2: int32, 0, 1024) {
          block([1024, 1024, tir.reduce_axis(0, 1024)], "C") as [m, n, k] {
            bind(m, i0)
            bind(n, i1)
            bind(k, i2)
            tir.reads([A[m, k], B[k, n]])
            tir.writes([C[m, n]])
            with init() {
              C[m, n] = 0f32
            }
            C[m, n] = (C[m, n] + (A[m, k]*B[k, n]))
        }
      }
    }
}

<class 'tvm.ir.module.IRModul

In [24]:
sch = tvm.tir.Schedule(ir_module)
print(sch)
block_c = sch.get_block("C")
(y, x, k) = sch.get_loops(block_c)
block_size = 64
yo, yi = sch.split(y, [None, block_size])
xo, xi = sch.split(x, [None, block_size])
sch.reorder(yo, xo, k, yi, xi)
print(sch.mod.script())

# ------- 编译 -------
func = tvm.build(sch.mod, target=target)
print(func.ir_module_by_target)
c = tvm.nd.array(np.empty((M, N), dtype=dtype), dev)
func(a, b, c)

evaluator = func.time_evaluator(func.entry_name, dev, number=number)
print("Baseline: %f" % evaluator(a, b, c).mean)

tir.Schedule(0x560442358bf8)
@tvm.script.ir_module
class Module:
    @tir.prim_func
    def func(A: tir.Buffer[(1024, 1024), "float32"], B: tir.Buffer[(1024, 1024), "float32"], C: tir.Buffer[(1024, 1024), "float32"]) -> None:
        # function attr dict
        tir.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with tir.block("root")
        for i0_0, i1_0, i2, i0_1, i1_1 in tir.grid(32, 32, 1024, 32, 32):
            with tir.block("C"):
                m = tir.axis.spatial(1024, i0_0 * 32 + i0_1)
                n = tir.axis.spatial(1024, i1_0 * 32 + i1_1)
                k = tir.axis.reduce(1024, i2)
                tir.reads(A[m, k], B[k, n])
                tir.writes(C[m, n])
                with tir.init():
                    C[m, n] = tir.float32(0)
                C[m, n] = C[m, n] + A[m, k] * B[k, n]
    
{llvm -keys=cpu -link-params=0: #[version = "0.0.5"]
@main = primfn(var_A: handle, var_B: handle, var_C: handle) -> ()
  attr = {"globa